In [2]:
# import os; os.chdir("/home/ubuntu/coma")
import os; os.chdir("/home/rodrigo/PhD/repos/pytorch_coma/")
import tensorflow as tf
import numpy as np
import ipywidgets as widgets
import yaml
from copy import deepcopy
import json
import re
import itertools
import pandas as pd
import time, datetime
from cardiac_mesh import CardiacMesh

## Some constant definitions

In [3]:
coma_dir = "/home/rodrigo/PhD/repos/pytorch_coma/"
yaml_subdir = os.path.join(coma_dir, "yamls")

## Data pre-processing
Generate two numpy files with meshes (for training and testing, respectively), from the corresponding VTK files.

## Load model parameters

We select a YAML file containing all the parameters necessary to define the model and the training process, as well as the paths where the input will be fetched and the output will be stored.
config.yaml is a reference yaml file, and other yaml files can be created from this one in the cell below

### Use the reference yaml to generate yamls with other parameters
This cell builds a set of yaml files for a grid of parameters, based on a reference yaml file. (If there are no new files to generate it can be commented out or skipped.)

In [4]:
config = yaml.safe_load(open(os.path.join(yaml_subdir, "config.yaml")))

data_files = {
    "LV": "/MULTIX/DATA/INPUT/disk_2/coma/Cardio/meshes/numpy_files/LV_all_subjects/train.npy",
    "LV_endo": "/MULTIX/DATA/INPUT/disk_2/coma/Cardio/meshes/numpy_files/LV_endo_all_subjects/LVED_all_subjects.npy",
    "RV": "/MULTIX/DATA/INPUT/disk_2/coma/Cardio/meshes/numpy_files/RV/RV_all.npy"
}

ids_files = {
    "LV": "/MULTIX/DATA/INPUT/disk_2/coma/Cardio/meshes/numpy_files/LV_all_subjects/LVED_all_subjects_subj_ids.txt",
    "LV_endo": "/MULTIX/DATA/INPUT/disk_2/coma/Cardio/meshes/numpy_files/LV_endo_all_subjects/LVED_all_subjects_subj_ids.txt",
    "RV": "/MULTIX/DATA/INPUT/disk_2/coma/Cardio/meshes/numpy_files/RV/RV_all_subject_ids.txt"
}

In [ ]:
config


In [5]:
# kk = np.load(data_files["LV"], allow_pickle=True)
# ids=[x.strip() for x in open(ids_files["RV"])]
# 
# pp = np.stack([k for k in kk if k.shape == (3478,3)]) #np.vstack([kk ])
# ids = [ids[i] for i,k in enumerate(kk) if k.shape == (3478,3)] #np.vstack([kk ])
# np.save(data_files["RV"]+"_", arr=pp)

In [6]:
# nTraining = [100, 200, 400, 800, 1600, 3200, 6400]
nTraining = [1600, 3200] #, 200, 400, 800, 1600, 3200, 6400]
# nTraining = [1600, 3200, 6400]
nTest = 2000

Fs = [
  # [16, 32, 32, 64],
  # [16, 32, 64, 128],
  # [8, 16, 32, 64],
  [8, 16, 32, 64, 128] #,
]

dir_pattern = "{}__ds_{}__nz_{}__reg_{}__lr_{}"
dss = [ 
  # [4, 4, 4, 2], 
  # [4, 4, 3, 2], 
  # [4, 4, 2, 2], 
  [2, 2, 2, 2, 2] #,
  # [4, 3, 3, 3]
] # downsampling factors

regs = [1e-4, 5e-4] # [1e-5, 1e-4, 1e-3]
nzs = [8] # [4, 8, 16, 32]
chambers = ["LV", "RV"]#, "LV_endo"]
nepochss = [500]
learning_rates = [1e-3, 1e-2] #, 5e-2]
decay_rates = [0.99] #, 0.995, 0.999]
losses = ['l1', 'l2']
batch_sizes = [16, 64]

parameters_grid = [
  dss, regs, nzs, chambers,
  nepochss, learning_rates, decay_rates,
  nTraining, losses, Fs
]

for comb in itertools.product(*parameters_grid):
    
    ds = comb[0]
    reg = comb[1]
    nz = comb[2]
    chamber = comb[3]
    nepochs = comb[4]
    learning_rate = comb[5]
    decay_rate = comb[6]
    nTr = comb[7]
    which_loss = comb[8]
    F = comb[9]
    
    config['num_epochs'] = nepochs
    config['ds_factors'] = ds
    config['partition'] = chamber
    config['regularization'] = reg
    config['learning_rate'] = learning_rate
    config['decay_rate'] = decay_rate
    config['nz'] = nz
    config['nTraining'] = nTr
    config['nVal_max'] = 200
    config['nVal_fraction'] = 0.20
    config['ids_file'] = ids_files[chamber]
    config['data_file'] = data_files[chamber]
    config['which_loss'] = which_loss
    config['F'] = F
    config['K'] = [6] * len(F)
    
    # config['data_dir'] = config['data_dir']/{}'.format(config['partition'])
  
    config['dir_name'] = dir_pattern.format(
                              chamber,
                              "".join([str(x) for x in ds]), 
                              nz, reg, learning_rate
                          )
    
    hsh = hash(tuple([config[k] for k in config.keys() if not isinstance(config[k], list)])) % 1000000
    config['hash'] = hsh
    yaml.dump(config, open(os.path.join(yaml_subdir, "config__{}__{}.yaml".format(config['dir_name'], hsh)), "w"))            

In [7]:
def to_tuple(x):
    return tuple(x) if isinstance(x, list) else x

def to_list(x):
    return list(x) if isinstance(x, tuple) else x

In [8]:
config_files = {x:yaml.safe_load(open(os.path.join(yaml_subdir, x))) for x in os.listdir(yaml_subdir) if x != "config.yaml"}
ref_conf = config_files[config_files.keys()[0]]

In [9]:
dif_cols = [ col for col in ref_conf.keys() if any([config_files[fn][col] != ref_conf[col] for fn in config_files]) ]

params_df = pd.DataFrame(
    [[config_files[x][col] for col in dif_cols] for x in config_files], 
    columns=dif_cols
)

dif_cols.remove("hash")
dif_cols.remove("dir_name")
dif_cols.remove("ids_file")
dif_cols.remove("data_file")

selection_w = {}
# for col in [ x for x in dif_cols if x != "hash" ]:

# params_df.loc[:,dif_cols]

In [10]:
for col in dif_cols:
  print(col)
  selection_w[col] = widgets.SelectMultiple(
      options=set([
          to_tuple(
              yaml.safe_load(
                 open(os.path.join(yaml_subdir, x))
              )[col]
          ) for x in config_files
      ]), 
      description=col)
  display(selection_w[col])

nTraining


SelectMultiple(description=u'nTraining', options=(1600, 3200), value=())

regularization


SelectMultiple(description=u'regularization', options=(0.0005, 0.0001), value=())

learning_rate


SelectMultiple(description=u'learning_rate', options=(0.001, 0.01), value=())

which_loss


SelectMultiple(description=u'which_loss', options=('l2', 'l1'), value=())

partition


SelectMultiple(description=u'partition', options=('LV', 'RV'), value=())

In [12]:
selected_values = {x: [to_list(y) for y in selection_w[x].value] for x in selection_w}

In [13]:
params_df.apply(axis=1, func = lambda row: [row[x] in selected_values[x] for x in selected_values])
yaml_files_df = params_df[params_df.apply(axis=1, func = lambda row: all([row[x] in selected_values[x] for x in selected_values]))]
yaml_files_df[dif_cols]

,nTraining,regularization,learning_rate,which_loss,partition
2,1600,0.0001,0.001,l1,LV
3,3200,0.0005,0.001,l1,LV
4,3200,0.0001,0.001,l1,RV
5,1600,0.0005,0.010,l1,RV
7,1600,0.0001,0.010,l1,RV
8,3200,0.0005,0.010,l1,LV
9,3200,0.0005,0.001,l1,RV
11,1600,0.0005,0.010,l1,LV
14,1600,0.0001,0.010,l1,LV
15,3200,0.0001,0.001,l1,LV


This block creates a text file containing the names of a set of YAML files select using the above widgets. This allow to select a series of sets of parameters describing the network architecture and its training process.

In [14]:
with open("runs_yamls.txt", "w") as ff:
    for row_index in range(yaml_files_df.shape[0]):                
        yaml_file = "config__{}__{}.yaml".format(
            yaml_files_df.iloc[row_index]['dir_name'], 
            yaml_files_df.iloc[row_index]['hash']
        )
        # print(yaml_file)
        ff.write("%s\n" % yaml_file)
    
#  for chamber in w_chamber.value:
#    for ds in w_ds.value:
#      for nz in w_nz.value:
#        for reg in w_reg.value:
#          for lr in w_lr.value:
#              yaml_file = "config__{}.yaml".format(dir_pattern).format(
#                           chamber, 
#                           ds.replace(",", ""), 
#                           nz, reg, lr)
#              yaml_file = os.path.join(yaml_subdir, yaml_file)


In [ ]:
params = yaml.safe_load(open("yamls/config__RV__ds_4422__nz_16__reg_0.0001__lr_0.001__730462.yaml"))
min(params['nVal_fraction'] * params['nTraining'], params['nVal_max'])

## Generate Tensorflow model

#### Path definitions

In [ ]:
output_dir = "/MULTIX/DATA/INPUT/disk_2/coma/Cardio/output/"
checkpoints_dir = "{}/checkpoints".format(output_dir)

In [ ]:
w_yaml = widgets.Dropdown(
    options= [
        x for x in os.listdir(yaml_subdir) 
          if x.endswith(".yaml")
            and x.startswith("config")
    ],
    description='Yaml File',
    disabled=False
)

w_yaml = widgets.Dropdown(
    options= [x.strip() for x in open("runs_yamls.txt").readlines()],
    description='Yaml File',
    disabled=False
)


display(w_yaml)

In [ ]:
params = yaml.safe_load(open(os.path.join(yaml_subdir, w_yaml.value)))
#params['num_epochs'] = 500
#params['nTraining'] = 1000
#params['nTest'] = 2000
#params['nVal'] = 100

In [ ]:
params

In [ ]:
from main import generate_model, run as train
train(params)

#### Select YAML file

In [ ]:
regex = re.compile("(.*)__ds_(.*)__nz_(.*)__reg_(.*)__lr_(.*)")
kk = [regex.match(x).groups() for x in os.listdir(checkpoints_dir)]
available_runs = ["config__{}__ds_{}__nz_{}__reg_{}__lr_{}".format(*k) for k in kk]

w_yaml = widgets.Dropdown(
    options= [x + ".yaml" for x in available_runs],
    description='Yaml File',
    disabled=False
)

display(w_yaml)

In [ ]:
config = yaml.safe_load(open(os.path.join(yaml_subdir, w_yaml.value)))

checkpoints_subdir = os.path.join(checkpoints_dir, config['dir_name'])

from main2 import generate_model, run as train

params = deepcopy(config)
model = generate_model(params)
model.p

### Fit the model
The cell below fits the model


## Restore session from last checkpoint

In [ ]:
sess = tf.Session()
last_chkpt = tf.train.latest_checkpoint(checkpoints_subdir)
new_saver = tf.train.import_meta_graph(last_chkpt + ".meta")
new_saver.restore(sess, last_chkpt)

data_prefix = "data/coma_cardiac/{}".format(config["partition"])
z_file_pattern = "{}/latent_space/{{}}_latent_base_{{}}.csv".format(output_dir)

io_mapping = { subset: {"data": os.path.join(data_prefix, "{}.npy".format(subset)),
                        "latent_space": z_file_pattern.format(config["dir_name"], subset)}
               for subset in ["train", "test"]
             }

for data_subset in io_mapping:
    meshes = np.load(io_mapping[data_subset]["data"])
    z = model.encode(data=meshes)
    reconstructed_meshes = model.decode(z)
    np.savetxt(io_mapping[data_subset]["latent_space"], z, delimiter=",")

# reconst_errors = np.linalg.norm(x=meshes-reconstructed_meshes, axis=2)

from VTK.VTKMesh import VTKObject as Mesh
mm = Mesh(filename="data/vtk_meshes/1000336/output.001.vtk")
mm = mm.extractSubpart(subparts_ids[config["partition"]])
mesh = Mesh()
mesh.points = reconstructed_meshes[1,:]
mesh.triangles = mm.triangles
mesh.SaveMeshToVTK("{}/vtk/{}.vtk".format(output_dir, config["dir_name"]))

# Analysis of model performance

In [ ]:
import vtkplotter
import k3d
k3d.mesh() 


In [ ]:
# import matplotlib.pyplot as plt4
# %load_ext rpy2.ipython
# %R library(tidyverse)
# z_train_file = io_mapping['train']['latent_space']
# z_test_file = io_mapping['test']['latent_space']
# %R -i z_train_file
# %R -i z_test_file
# %R df <- read.csv(z_train_file, header = FALSE) %>% mutate(subset="train")
# %R df <- rbind(df, read.csv(z_test_file, header = FALSE) %>% mutate(subset="test"))
# %R colnames(df) <- c(sapply(X = 1:8, FUN = function(x) paste0("z", x)), "subset")
# 
# %R df <- df %>% gather(key = "latent_comp", value = "value", -subset) %>% filter(latent_comp != "z3" & latent_comp != "z5")
# %R df <- df %>% group_by(latent_comp) %>% mutate(value = scale(value, center = TRUE, scale = TRUE))
# %R pp <- ggplot(df, mapping = aes(value, stat(density)))
# %R pp <- pp + theme_bw() + geom_histogram()
# %R pp <- pp + facet_grid(rows = vars(latent_comp), cols=vars(subset), scales = "free")
# %R pp